In [1]:
import configparser
import requests
import pandas as pd

In [2]:
# read market stack api key
config = configparser.ConfigParser()
config.read('config.ini')
api_key = config['marketstack']['api_key']

In [4]:
# how many days of data?
daylimit = 30

In [5]:
# make a request to the marketstack API to extract stock prices of Tesla
url = 'http://api.marketstack.com/v1/tickers/tsla/eod'

params = {
    'access_key' : api_key,
    'limit': daylimit,
}

response = requests.get(url, params)

 # check the request status
if response.status_code != 200:
    raise Exception("Request returned an error: {} {}".format(response.status_code, response.text))

In [6]:
# change response to json format
response_json = response.json()
response_json.keys()

dict_keys(['pagination', 'data'])

In [8]:
# check API response related to paginaion 
response_json['pagination']

{'limit': 30, 'offset': 0, 'count': 30, 'total': 251}

In [9]:
# check a sample data
response_json['data']['eod'][0]

{'open': 257.78,
 'high': 268.94,
 'low': 257.65,
 'close': 263.62,
 'volume': 122656030.0,
 'adj_high': 268.94,
 'adj_low': 257.65,
 'adj_close': 263.62,
 'adj_open': 257.75,
 'adj_volume': 122656030.0,
 'split_factor': 1.0,
 'dividend': 0.0,
 'symbol': 'TSLA',
 'exchange': 'XNAS',
 'date': '2023-10-10T00:00:00+0000'}

In [10]:
# define a function to convert api outout to pandas dataframe
def json_eods_torows(json_eods): 
    # create a list for all entries
    df_rows = [] 
    for item in json_eods:
        row_dict = {}
        row_dict['open'] = item['open']
        row_dict['high'] = item['high']
        row_dict['low'] = item['low']
        row_dict['close'] = item['close']
        row_dict['volume'] = item["volume"]
        row_dict['split_factor'] = item['split_factor']
        row_dict['dividend'] = item['dividend']
        row_dict['symbol'] = item['symbol']
        row_dict['date'] = item['date']
        df_rows.append(row_dict)
    return df_rows

In [11]:
df = pd.DataFrame(response_json['data']['eod'])
df.head()

,open,high,low,close,volume,adj_high,adj_low,adj_close,adj_open,adj_volume,split_factor,dividend,symbol,exchange,date
0,257.78,268.94,257.65,263.62,122656030.0,268.94,257.65,263.62,257.75,122656030.0,1.0,0.0,TSLA,XNAS,2023-10-10T00:00:00+0000
1,255.21,261.36,252.05,259.67,101377947.0,261.36,252.05,259.67,255.31,101377947.0,1.0,0.0,TSLA,XNAS,2023-10-09T00:00:00+0000
2,253.98,261.65,250.65,260.53,118121812.0,261.65,250.65,260.53,253.98,118121812.0,1.0,0.0,TSLA,XNAS,2023-10-06T00:00:00+0000
3,260.00,263.60,256.25,260.05,119159214.0,263.60,256.25,260.05,260.00,119159214.0,1.0,0.0,TSLA,XNAS,2023-10-05T00:00:00+0000
4,248.14,261.86,247.60,261.16,129721567.0,261.86,247.60,261.16,248.14,129721567.0,1.0,0.0,TSLA,XNAS,2023-10-04T00:00:00+0000


In [12]:
# check if there is any missing values and data types
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 15 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   open          30 non-null     float64
 1   high          30 non-null     float64
 2   low           30 non-null     float64
 3   close         30 non-null     float64
 4   volume        30 non-null     float64
 5   adj_high      30 non-null     float64
 6   adj_low       30 non-null     float64
 7   adj_close     30 non-null     float64
 8   adj_open      30 non-null     float64
 9   adj_volume    30 non-null     float64
 10  split_factor  30 non-null     float64
 11  dividend      30 non-null     float64
 12  symbol        30 non-null     object 
 13  exchange      30 non-null     object 
 14  date          30 non-null     object 
dtypes: float64(12), object(3)
memory usage: 3.6+ KB


In [13]:
# create a date column
df['datetime'] = pd.to_datetime(df['date'])
df['date'] = df['datetime'].dt.strftime('%Y-%m-%d')
df['date'] = pd.to_datetime(df['date'])
df.head()

,open,high,low,close,volume,adj_high,adj_low,adj_close,adj_open,adj_volume,split_factor,dividend,symbol,exchange,date,datetime
0,257.78,268.94,257.65,263.62,122656030.0,268.94,257.65,263.62,257.75,122656030.0,1.0,0.0,TSLA,XNAS,2023-10-10,2023-10-10 00:00:00+00:00
1,255.21,261.36,252.05,259.67,101377947.0,261.36,252.05,259.67,255.31,101377947.0,1.0,0.0,TSLA,XNAS,2023-10-09,2023-10-09 00:00:00+00:00
2,253.98,261.65,250.65,260.53,118121812.0,261.65,250.65,260.53,253.98,118121812.0,1.0,0.0,TSLA,XNAS,2023-10-06,2023-10-06 00:00:00+00:00
3,260.00,263.60,256.25,260.05,119159214.0,263.60,256.25,260.05,260.00,119159214.0,1.0,0.0,TSLA,XNAS,2023-10-05,2023-10-05 00:00:00+00:00
4,248.14,261.86,247.60,261.16,129721567.0,261.86,247.60,261.16,248.14,129721567.0,1.0,0.0,TSLA,XNAS,2023-10-04,2023-10-04 00:00:00+00:00


In [14]:
# check data type again
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 16 columns):
 #   Column        Non-Null Count  Dtype              
---  ------        --------------  -----              
 0   open          30 non-null     float64            
 1   high          30 non-null     float64            
 2   low           30 non-null     float64            
 3   close         30 non-null     float64            
 4   volume        30 non-null     float64            
 5   adj_high      30 non-null     float64            
 6   adj_low       30 non-null     float64            
 7   adj_close     30 non-null     float64            
 8   adj_open      30 non-null     float64            
 9   adj_volume    30 non-null     float64            
 10  split_factor  30 non-null     float64            
 11  dividend      30 non-null     float64            
 12  symbol        30 non-null     object             
 13  exchange      30 non-null     object             
 14  date        

In [15]:
# sort by date ascendingly
df.sort_values(by = ['date'])

,open,high,low,close,volume,adj_high,adj_low,adj_close,adj_open,adj_volume,split_factor,dividend,symbol,exchange,date,datetime
29,238.580,257.4800,237.7700,257.18,134047603.0,257.4800,237.7700,257.18,238.580,134047603.0,1.0,0.0,TSLA,XNAS,2023-08-29,2023-08-29 00:00:00+00:00
28,254.200,260.5100,250.5900,256.90,121988437.0,260.5100,250.5900,256.90,254.200,121988437.0,1.0,0.0,TSLA,XNAS,2023-08-30,2023-08-30 00:00:00+00:00
27,255.980,261.1800,255.0500,258.08,108861698.0,261.1800,255.0500,258.08,255.980,108861698.0,1.0,0.0,TSLA,XNAS,2023-08-31,2023-08-31 00:00:00+00:00
26,257.260,259.0794,242.0100,245.01,132541640.0,259.0794,242.0100,245.01,257.260,132541640.0,1.0,0.0,TSLA,XNAS,2023-09-01,2023-09-01 00:00:00+00:00
25,245.000,258.0000,244.8600,256.49,129469565.0,258.0000,244.8600,256.49,245.000,129469565.0,1.0,0.0,TSLA,XNAS,2023-09-05,2023-09-05 00:00:00+00:00
24,255.135,255.3900,245.0600,251.92,116959759.0,255.3900,245.0600,251.92,255.135,116959759.0,1.0,0.0,TSLA,XNAS,2023-09-06,2023-09-06 00:00:00+00:00
23,245.070,252.8100,243.2650,251.49,115312886.0,252.8100,243.2650,251.49,245.070,115312886.0,1.0,0.0,TSLA,XNAS,2023-09-07,2023-09-07 00:00:00+00:00
22,251.220,256.5200,246.6700,248.50,118559635.0,256.5200,246.6700,248.50,251.220,118559635.0,1.0,0.0,TSLA,XNAS,2023-09-08,2023-09-08 00:00:00+00:00
21,264.270,274.8500,260.6100,273.58,174667852.0,274.8500,260.6100,273.58,264.270,174667852.0,1.0,0.0,TSLA,XNAS,2023-09-11,2023-09-11 00:00:00+00:00
20,270.760,278.3900,266.6000,267.48,135999866.0,278.3900,266.6000,267.48,270.760,135999866.0,1.0,0.0,TSLA,XNAS,2023-09-12,2023-09-12 00:00:00+00:00


In [21]:
# save out the stock price data
df.to_csv('../data/Tesla_stock.csv', index = False)